In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1596479/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_1596479/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_1_3,0.752522,0.064642,-1.409749,0.853332,0.413699,1.875103,1.776980,0.199954,1.988604e+14,0.643194,0.489078,0.653324,67.765234,139.520014,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_1_2,0.747791,0.062340,-1.413680,0.848081,0.421607,1.879717,1.779879,0.207112,2.004606e+14,0.649313,0.479311,0.659539,67.727361,139.482141,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_1_0,0.737711,0.050830,-1.444431,0.840203,0.438458,1.902791,1.802555,0.217852,2.036435e+14,0.662162,0.458500,0.672590,67.648981,139.403761,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_2_9,0.842094,0.043083,-1.392162,0.824274,0.263964,1.918321,1.764011,0.232323,1.573055e+14,0.513775,0.674002,0.521866,68.663883,140.418662,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_5_9,0.922705,0.041496,-1.342579,0.855831,0.129212,1.921504,1.727448,0.368117,1.364273e+14,0.359460,0.840422,0.365121,70.092607,141.847387,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,model_3_13_3,0.991277,-0.654972,-2.184126,0.987384,0.014581,3.317705,2.348015,0.012916,1.377042e+13,0.120752,0.962784,0.122654,106.456058,213.001035,"Hidden Size=[12], regularizer=0.02, learning_r..."
533,model_3_13_2,0.990870,-0.704480,-2.191964,0.988668,0.015262,3.416953,2.353796,0.011602,1.434615e+13,0.123538,0.961047,0.125484,106.364819,212.909795,"Hidden Size=[12], regularizer=0.02, learning_r..."
534,model_3_12_4,0.987563,-0.784030,-2.110583,0.980715,0.020791,3.576425,2.293784,0.046311,2.450454e+13,0.144190,0.946935,0.146461,105.746503,212.291479,"Hidden Size=[12], regularizer=0.02, learning_r..."
535,model_3_11_9,0.984245,-1.143630,-2.115651,0.968338,0.026337,4.297313,2.297522,0.047478,3.283938e+13,0.162286,0.932779,0.164842,105.273577,211.818553,"Hidden Size=[12], regularizer=0.02, learning_r..."
